In [1]:
""" 3/27/18 
With a functional mister_ed package, it's time to run the madry challenge.

Step 1 is to build a defended network that can transfer to their tensorflow nets.
Because Madry uses tensorflow, transferability is a must. 

From Madry:
The model is a residual convolutional neural network consisting of five residual units and a fully connected layer. 
This architecture is derived from the "w32-10 wide" variant of the Tensorflow model repository. 
The network was trained against an iterative adversary that is allowed to perturb each pixel by at most epsilon=8.0.
"""
None

In [2]:
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn

import config
import prebuilt_loss_functions as plf
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain


In [3]:
# Step -1: load the classifier, data_loader, and the normalizer 
use_gpu = True
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
classifier_net.eval()

val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)


Files already downloaded and verified


In [4]:
# Step 0: initialize hyperparams for attack and training 
PGD_L_INF = 8.0 / 255.0
PGD_STEP_SIZE = 1.0 / 255.0
PGD_NUM_ITER = 16
PGD_TRAINING_PROP = 1.0

In [5]:
# Step 1: setup loss object 
standard_xentropy = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer)

In [6]:
# Step 2: build attack object and its parameters
pgd_attack_object = aa.LInfPGD(classifier_net, cifar_normer, standard_xentropy)
pgd_attack_params = advtrain.AdversarialAttackParameters(
                            pgd_attack_object, 
                            PGD_TRAINING_PROP,
                            {'attack_kwargs': 
                                {'l_inf_bound': PGD_L_INF,
                                 'step_size': PGD_STEP_SIZE,
                                 'num_iterations': PGD_NUM_ITER,
                                 'random_init': True,
                                 'signed': True,
                                 'verbose': False}})


In [7]:
# Step 3: build training object, training loss, data loader

pgd_cifar_training = advtrain.AdversarialTraining(classifier_net, 
                                                  cifar_normer, 
                                                  'full_pgd_cifar_madry_params',
                                                  'cifar_resnet32')
train_loss = nn.CrossEntropyLoss()
train_loader = cifar_loader.load_cifar_data('train', normalize=False)



Files already downloaded and verified


In [ ]:
# Step 4: Do the training 
pgd_cifar_training.train(train_loader, 256, train_loss, 
                         attack_parameters=pgd_attack_params,
                         use_gpu=True, verbosity='high')

[1,   100] accuracy: (9.375, 67.969)
[1,   100] loss: 0.164
[1,   200] accuracy: (17.188, 71.875)
[1,   200] loss: 0.084
[1,   300] accuracy: (17.969, 65.625)
[1,   300] loss: 0.077
[2,   100] accuracy: (30.469, 76.562)
[2,   100] loss: 0.071
[2,   200] accuracy: (26.562, 82.031)
[2,   200] loss: 0.068
[2,   300] accuracy: (32.031, 75.000)
[2,   300] loss: 0.068
COMPLETED EPOCH 0001... checkpointing here
[3,   100] accuracy: (30.469, 71.875)
[3,   100] loss: 0.065
[3,   200] accuracy: (29.688, 75.000)
[3,   200] loss: 0.065
[3,   300] accuracy: (28.125, 73.438)
[3,   300] loss: 0.064
COMPLETED EPOCH 0002... checkpointing here
[4,   100] accuracy: (36.719, 86.719)
[4,   100] loss: 0.063
[4,   200] accuracy: (34.375, 82.812)
[4,   200] loss: 0.062
[4,   300] accuracy: (33.594, 74.219)
[4,   300] loss: 0.062
COMPLETED EPOCH 0003... checkpointing here
[5,   100] accuracy: (26.562, 78.125)
[5,   100] loss: 0.060
[5,   200] accuracy: (39.062, 84.375)
[5,   200] loss: 0.060
[5,   300] accurac

COMPLETED EPOCH 0035... checkpointing here
[37,   100] accuracy: (42.969, 92.188)
[37,   100] loss: 0.046
[37,   200] accuracy: (43.750, 82.812)
[37,   200] loss: 0.047
[37,   300] accuracy: (33.594, 84.375)
[37,   300] loss: 0.047
COMPLETED EPOCH 0036... checkpointing here
[38,   100] accuracy: (40.625, 85.938)
[38,   100] loss: 0.047
[38,   200] accuracy: (45.312, 89.844)
[38,   200] loss: 0.047
[38,   300] accuracy: (37.500, 85.938)
[38,   300] loss: 0.047
COMPLETED EPOCH 0037... checkpointing here
[39,   100] accuracy: (44.531, 89.844)
[39,   100] loss: 0.046
[39,   200] accuracy: (46.094, 88.281)
[39,   200] loss: 0.046
[39,   300] accuracy: (52.344, 91.406)
[39,   300] loss: 0.047
COMPLETED EPOCH 0038... checkpointing here
[40,   100] accuracy: (45.312, 89.844)
[40,   100] loss: 0.046
[40,   200] accuracy: (42.188, 83.594)
[40,   200] loss: 0.046
[40,   300] accuracy: (40.625, 82.031)
[40,   300] loss: 0.047
COMPLETED EPOCH 0039... checkpointing here
[41,   100] accuracy: (46.094

[72,   200] accuracy: (57.031, 92.969)
[72,   200] loss: 0.042
[72,   300] accuracy: (50.000, 88.281)
[72,   300] loss: 0.043
COMPLETED EPOCH 0071... checkpointing here
[73,   100] accuracy: (48.438, 89.062)
[73,   100] loss: 0.042
[73,   200] accuracy: (42.188, 89.062)
[73,   200] loss: 0.043
[73,   300] accuracy: (40.625, 86.719)
[73,   300] loss: 0.043
COMPLETED EPOCH 0072... checkpointing here
[74,   100] accuracy: (42.969, 86.719)
[74,   100] loss: 0.042
[74,   200] accuracy: (46.875, 84.375)
[74,   200] loss: 0.042
[74,   300] accuracy: (45.312, 88.281)
[74,   300] loss: 0.043
COMPLETED EPOCH 0073... checkpointing here
[75,   100] accuracy: (42.188, 89.844)
[75,   100] loss: 0.041
[75,   200] accuracy: (49.219, 90.625)
[75,   200] loss: 0.043
[75,   300] accuracy: (46.875, 87.500)
[75,   300] loss: 0.042
COMPLETED EPOCH 0074... checkpointing here
[76,   100] accuracy: (46.875, 88.281)
[76,   100] loss: 0.042
[76,   200] accuracy: (45.312, 85.938)
[76,   200] loss: 0.042
[76,   30